#### Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:

* tracking server: yes, local server
* backend store: sqlite database
* artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

```mlflow server --backend-store-uri sqlite:///backend.db```

In [16]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:8000")

In [17]:
print(f"tracking uri : '{mlflow.get_tracking_uri()}'")

tracking uri : 'http://127.0.0.1:8000'


In [18]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: './mlruns/1/ef03b4d0d9cc4d92b7e9f5057feaa575/artifacts'


In [20]:
mlflow.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

#### Interacting with the model registry

In [21]:
from mlflow.tracking import MlflowClient

client = MlflowClient('http://127.0.0.1:8000')

In [22]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654174378086, description='', last_updated_timestamp=1654241010502, latest_versions=[<ModelVersion: creation_timestamp=1654241010502, current_stage='None', description='', last_updated_timestamp=1654241010502, name='iris-classifier', run_id='460cf9a46b764e8a84bb2c6f0ed55343', run_link='', source='./mlruns/1/460cf9a46b764e8a84bb2c6f0ed55343/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>], name='iris-classifier', tags={}>]

In [23]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
print(run_id)

ef03b4d0d9cc4d92b7e9f5057feaa575


In [24]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2022/06/03 13:04:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 3
Created version '3' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654241674581, current_stage='None', description='', last_updated_timestamp=1654241674581, name='iris-classifier', run_id='ef03b4d0d9cc4d92b7e9f5057feaa575', run_link='', source='./mlruns/1/ef03b4d0d9cc4d92b7e9f5057feaa575/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>

In [25]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654174378086, description='', last_updated_timestamp=1654241674581, latest_versions=[<ModelVersion: creation_timestamp=1654241674581, current_stage='None', description='', last_updated_timestamp=1654241674581, name='iris-classifier', run_id='ef03b4d0d9cc4d92b7e9f5057feaa575', run_link='', source='./mlruns/1/ef03b4d0d9cc4d92b7e9f5057feaa575/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>], name='iris-classifier', tags={}>]